In [1]:
import pandas as pd

# Import q_stats

In [2]:
qstats=pd.read_csv('../data/qstats_for_M1')
qstats

,Unnamed: 0,content_id,bundle_id,correct_answer,part,tags,qstats_count,qstats_answered_correctly,qstats_prior_question_had_explanation,qstats_answered_correctly_knowing_having_had_explanation,qstats_answered_correctly_knowing_having_not_had_explanation
0,0,0,0,0,1,51 131 162 38,6903,0.907721,0.948138,0.911994,0.829609
1,1,1,1,1,1,131 36 81,7398,0.890646,0.980400,0.892183,0.813793
2,2,2,2,0,1,131 101 162 92,44905,0.554281,0.888253,0.562213,0.491232
3,3,3,3,0,1,131 149 162 29,22973,0.779437,0.958473,0.783460,0.686583
4,4,4,4,3,1,131 5 162 38,31736,0.613215,0.530313,0.654308,0.566819
...,...,...,...,...,...,...,...,...,...,...,...
13518,13518,13518,13518,3,5,14,868,0.786866,0.998848,0.786621,1.000000
13519,13519,13519,13519,3,5,8,924,0.571429,0.994589,0.570185,0.800000
13520,13520,13520,13520,2,5,73,861,0.672474,0.995354,0.670945,1.000000
13521,13521,13521,13521,0,5,125,870,0.808046,0.996552,0.808535,0.666667


# Model import

In [3]:
import pickle
my_pipeline = pickle.load(open('../models/xgboost_pipe_M1.pkl',"rb"))  ## wb= write bites : le b est important

In [4]:
pipeline_features_list=pd.read_csv('../models/xgboost_pipe_M1_features_list')

In [5]:
[pipeline_features_list.feature.to_list()]

[['user_personalized_qstat_knowing_had_explanation_or_not',
  'user_avg_score_cum_on_this_part',
  'user_correct_answers_cum_on_this_part',
  'qstats_answered_correctly',
  'user_avg_score_cum',
  'qstats_prior_question_had_explanation',
  'already_seen',
  'user_activity_cumcount',
  'at_least_one_lesson']]

# Initialisation utilisateur débutant

In [6]:
user_history_empty=pd.DataFrame({#following columns are the impute of each loop
                             ### TO BE IMPUTED ###
                             'content_id':[-1],
                             'content_type_id':[-1],
                             'prior_question_had_explanation':False,
                             # following columns depend of previous history of the user : 
                             ### TO BE UPDATED WHATEVER THE CONTENT_TYPE ###
                             'user_activity_cumcount':[-1],
                             'at_least_one_lesson':[0],
                             ### TO BE UPDATED IF LAST WAS QUESTION , ELSE COPIED ###
                             'user_avg_score_cum':[0.499],
                             'user_correct_answers_cum':[0],
                             'user_avg_score_cum_part1':[0.499],
                             'user_avg_score_cum_part2':[0.499],
                             'user_avg_score_cum_part3':[0.499],
                             'user_avg_score_cum_part4':[0.499],
                             'user_avg_score_cum_part5':[0.499],
                             'user_avg_score_cum_part6':[0.499],
                             'user_avg_score_cum_part7':[0.499],
                             'user_correct_answers_cum_part1':[0],
                             'user_correct_answers_cum_part2':[0],
                             'user_correct_answers_cum_part3':[0],
                             'user_correct_answers_cum_part4':[0],
                             'user_correct_answers_cum_part5':[0],
                             'user_correct_answers_cum_part6':[0],
                             'user_correct_answers_cum_part7':[0],
                             # following columns are pure question stats : 
                             ### TO BE IMPORTED ###
                             'part':[-1],
                             'qstats_answered_correctly':[-1],
                             'qstats_prior_question_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_not_had_explanation':[-1],
                             # following columns depend of the current question AND the hisory of user
                             ### TO BE COMPUTED ###
                             'user_personalized_qstat_knowing_had_explanation_or_not':[-1],
                             'already_seen':[-1],
                             'user_avg_score_cum_on_this_part':[-1],
                             'user_correct_answers_cum_on_this_part':[-1],
                             # the following line is the prediction to be made
                             ### TO BE PREDICTED ###
                             'answered_correctly':[-1]
                          })
user_history

NameError: name 'user_history' is not defined

# User history update fonction

In [19]:
def user_history_update(content_type_id,
                        content_id,
                        data_qstats,
                        user_history=None,
                        mode='training',    #autre choix : mode exam
                        prior_question_had_explanation=False):
    '''Crée ou met à jour l'hisorique d'un utilisateur, stockée dans un df'''

    if not type(user_history)==pd.DataFrame:
        user_history=pd.DataFrame({
                             #following columns are the impute of each loop
                             ### TO BE IMPUTED ###
                             'content_id':[-1],
                             'content_type_id':[-1],
                             'prior_question_had_explanation':False,
                             'mode':'n/a',
                             # following columns depend of previous history of the user : 
                             ### TO BE UPDATED WHATEVER THE CONTENT_TYPE ###
                             'user_activity_cumcount':[-1],
                             ### TO BE UPDATED IF LAST WAS LECTURE ###
                             'at_least_one_lesson':[0],
                             ### TO BE UPDATED IF LAST WAS QUESTION ###
                             'user_avg_score_cum':[0.499],
                             'user_correct_answers_cum':[0],
                             'user_avg_score_cum_part1':[0.499],
                             'user_avg_score_cum_part2':[0.499],
                             'user_avg_score_cum_part3':[0.499],
                             'user_avg_score_cum_part4':[0.499],
                             'user_avg_score_cum_part5':[0.499],
                             'user_avg_score_cum_part6':[0.499],
                             'user_avg_score_cum_part7':[0.499],
                             'user_correct_answers_cum_part1':[0],
                             'user_correct_answers_cum_part2':[0],
                             'user_correct_answers_cum_part3':[0],
                             'user_correct_answers_cum_part4':[0],
                             'user_correct_answers_cum_part5':[0],
                             'user_correct_answers_cum_part6':[0],
                             'user_correct_answers_cum_part7':[0],
                             # following columns are pure question stats : 
                             ### TO BE IMPORTED FROM QUESTIONS ###
                             'part':[-1],
                             'qstats_answered_correctly':[-1],
                             'qstats_prior_question_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_not_had_explanation':[-1],
                             # following columns depend of the current question AND the hisory of user
                             ### TO BE COMPUTED ###
                             'user_personalized_qstat_knowing_had_explanation_or_not':[-1],
                             'already_seen':[-1],
                             'user_avg_score_cum_on_this_part':[-1],
                             'user_correct_answers_cum_on_this_part':[-1],
                             # the following line is the prediction to be made
                             ### TO BE PREDICTED ###
                             'answered_correctly':[-1]
                          })
    

    last_line=user_history.iloc[-1]
    new_line =last_line.copy()
    
    last_content_type_id=user_history.iloc[-1]['content_type_id']
    
    ### TO BE IMPUTED ###
    new_line['content_id']=content_id
    new_line['content_type_id']=content_type_id
    new_line['prior_question_had_explanation']=prior_question_had_explanation
    new_line['mode']=mode
    ### TO BE UPDATED WHATEVER THE CONTENT_TYPE ###
    new_line['user_activity_cumcount'] += 1
    
    if mode=='training':
        if last_content_type_id==0:
            part=last_line['part']
            ### TO BE UPDATED IF LAST WAS QUESTION ###
            new_line['user_correct_answers_cum'] = last_line['user_correct_answers_cum']\
                                                 + last_line['answered_correctly']
            new_user_questions_count             = last_line['user_correct_answers_cum']\
                                                 / last_line['user_avg_score_cum']\
                                                 + 1
            new_line['user_avg_score_cum']       = new_line['user_correct_answers_cum']\
                                                 / new_user_questions_count

            new_line[f'user_correct_answers_cum_part{part}'] = last_line[f'user_correct_answers_cum_part{part}']\
                                                             + last_line['answered_correctly']
            vars()[f'new_user_questions_count_part{part}']   = last_line[f'user_correct_answers_cum_part{part}']\
                                                             / last_line[f'user_avg_score_cum_part{part}']\
                                                             + 1
            new_line[f'user_avg_score_cum_part{part}']       = new_line[f'user_correct_answers_cum_part{part}']\
                                                             / vars()[f'new_user_questions_count_part{part}']

        elif last_content_type_id==1:
            ### TO BE UPDATED IF LAST WAS LECTURE ###
            new_line['at_least_one_lesson']=1
        
    if content_type_id==0:        
        currect_question_stats=qstats.loc[qstats.content_id==content_id].iloc[-1]
        ### TO BE IMPORTED FROM QUESTIONS ###
        new_line['part']\
              = currect_question_stats['part']
        new_line['qstats_answered_correctly']\
              = currect_question_stats['qstats_answered_correctly']
        new_line['qstats_prior_question_had_explanation']\
              = currect_question_stats['qstats_prior_question_had_explanation']
        new_line['qstats_answered_correctly_knowing_having_had_explanation']\
              = currect_question_stats['qstats_answered_correctly_knowing_having_had_explanation']
        new_line['qstats_answered_correctly_knowing_having_not_had_explanation']\
              = currect_question_stats['qstats_answered_correctly_knowing_having_not_had_explanation']
        ### TO BE COMPUTED ###
        new_line['user_personalized_qstat_knowing_had_explanation_or_not']\
              = new_line['qstats_answered_correctly_knowing_having_had_explanation']\
             if prior_question_had_explanation\
           else new_line['qstats_answered_correctly_knowing_having_not_had_explanation']
        new_line['already_seen']\
              = 1 if content_id in user_history.loc[user_history.content_type_id==0,'content_id']\
           else 0
        new_line['user_avg_score_cum_on_this_part']=new_line[f'user_avg_score_cum_part{new_line["part"]}']
        new_line['user_correct_answers_cum_on_this_part']=new_line[f'user_correct_answers_cum_part{new_line["part"]}']
                                              
    elif content_type_id==1:
        
        ### TO BE IMPORTED ###
        new_line['part']= -1 # TODO : si on veut utiliser la partie de la lecture, il faut importer la base des lectures
        new_line['qstats_answered_correctly']= -1
        new_line['qstats_prior_question_had_explanation']= -1
        new_line['qstats_answered_correctly_knowing_having_had_explanation']= -1
        new_line['qstats_answered_correctly_knowing_having_not_had_explanation']= -1
        ### TO BE COMPUTED ###
        new_line['user_personalized_qstat_knowing_had_explanation_or_not']= -1
        new_line['already_seen']= -1
        new_line['user_avg_score_cum_on_this_part']= -1
        new_line['user_correct_answers_cum_on_this_part']= -1
        
    ### TO BE PREDICTED ###
    new_line['answered_correctly']= -1
                                
    user_history=user_history.append(new_line,ignore_index=True)

    return user_history

## Test de la fonction

In [20]:
user_history=user_history_update(0, #content_type_id=0 <=> question
                                 13522, #numéro d'une question au hasard
                                 qstats, #df des stats questions
                                 user_history=None, #None pour une utilisateur nouveau
                                 prior_question_had_explanation=False, #
                                )
user_history.iloc[-1,-1]=0.8
user_history

,content_id,content_type_id,prior_question_had_explanation,mode,user_activity_cumcount,at_least_one_lesson,user_avg_score_cum,user_correct_answers_cum,user_avg_score_cum_part1,user_avg_score_cum_part2,...,part,qstats_answered_correctly,qstats_prior_question_had_explanation,qstats_answered_correctly_knowing_having_had_explanation,qstats_answered_correctly_knowing_having_not_had_explanation,user_personalized_qstat_knowing_had_explanation_or_not,already_seen,user_avg_score_cum_on_this_part,user_correct_answers_cum_on_this_part,answered_correctly
0,-1,-1,False,n/a,-1,0,0.499,0,0.499,0.499,...,-1,-1.000000,-1.00000,-1.000000,-1.0,-1.0,-1,-1.000,-1,-1.0
1,13522,0,False,training,0,0,0.499,0,0.499,0.499,...,5,0.913953,0.99186,0.913247,1.0,1.0,0,0.499,0,0.8


In [21]:
user_history=user_history_update(0,13521,qstats,user_history,prior_question_had_explanation=False)
user_history

,content_id,content_type_id,prior_question_had_explanation,mode,user_activity_cumcount,at_least_one_lesson,user_avg_score_cum,user_correct_answers_cum,user_avg_score_cum_part1,user_avg_score_cum_part2,...,part,qstats_answered_correctly,qstats_prior_question_had_explanation,qstats_answered_correctly_knowing_having_had_explanation,qstats_answered_correctly_knowing_having_not_had_explanation,user_personalized_qstat_knowing_had_explanation_or_not,already_seen,user_avg_score_cum_on_this_part,user_correct_answers_cum_on_this_part,answered_correctly
0,-1,-1,False,n/a,-1,0,0.499,0.0,0.499,0.499,...,-1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1.000,-1.0,-1.0
1,13522,0,False,training,0,0,0.499,0.0,0.499,0.499,...,5,0.913953,0.991860,0.913247,1.000000,1.000000,0,0.499,0.0,0.8
2,13521,0,False,training,1,0,0.800,0.8,0.499,0.499,...,5,0.808046,0.996552,0.808535,0.666667,0.666667,0,0.800,0.8,-1.0


# Loop

In [15]:
loop_length=100
question_selection_strategy='random'

In [33]:
import random

### Initialize the user to None or starting level ###
user_history=None

for i in range(loop_length):
    ### CHOIX DE LA QUESTION ###
    if question_selection_strategy=='random':
        next_question_id=random.choice(qstats.content_id.to_list())
    
    
    user_history=user_history_update(0,
                                     next_question_id,
                                     qstats,
                                     user_history,
                                     mode='training',
                                     prior_question_had_explanation=random.uniform(0, 1)>0.1)
    
    ### PREDICTION ###
    user_history.iloc[-1,-1]\
        = my_pipeline.predict_proba(user_history[pipeline_features_list.feature.to_list()].iloc[-2:-1])[0,1]
user_history
    

,content_id,content_type_id,prior_question_had_explanation,mode,user_activity_cumcount,at_least_one_lesson,user_avg_score_cum,user_correct_answers_cum,user_avg_score_cum_part1,user_avg_score_cum_part2,...,part,qstats_answered_correctly,qstats_prior_question_had_explanation,qstats_answered_correctly_knowing_having_had_explanation,qstats_answered_correctly_knowing_having_not_had_explanation,user_personalized_qstat_knowing_had_explanation_or_not,already_seen,user_avg_score_cum_on_this_part,user_correct_answers_cum_on_this_part,answered_correctly
0,-1,-1,False,n/a,-1,0,0.499000,0.000000,0.499000,0.499000,...,-1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.000000
1,224,0,False,training,0,0,0.499000,0.000000,0.499000,0.499000,...,2,0.807905,0.977678,0.809908,0.720183,0.720183,0,0.499000,0.000000,0.070943
2,914,0,True,training,1,0,0.070943,0.070943,0.499000,0.070943,...,2,0.604541,0.984122,0.605624,0.537383,0.605624,0,0.070943,0.070943,0.733975
3,4450,0,False,training,2,0,0.402459,0.804918,0.499000,0.402459,...,5,0.699502,0.983891,0.700583,0.633508,0.633508,0,0.499000,0.000000,0.481057
4,7434,0,True,training,3,0,0.428658,1.285975,0.499000,0.402459,...,7,0.536606,0.965719,0.537638,0.507538,0.537638,0,0.499000,0.000000,0.555386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,10161,0,True,training,95,0,0.709098,67.364333,0.669345,0.766332,...,6,0.884046,0.964232,0.883368,0.902326,0.883368,0,0.771834,6.946505,0.839581
97,6648,0,True,training,96,0,0.710457,68.203914,0.669345,0.766332,...,6,0.537757,0.901602,0.539340,0.523256,0.539340,0,0.778609,7.786086,0.927608
98,2446,0,True,training,97,0,0.712696,69.131522,0.669345,0.766332,...,3,0.809124,0.970353,0.809710,0.789954,0.809710,0,0.638235,5.744112,0.573046
99,247,0,True,training,98,0,0.711271,69.704568,0.669345,0.766332,...,2,0.705328,0.982781,0.705895,0.672956,0.705895,0,0.766332,12.261313,0.840165


In [35]:
def TOEIC_scoring (user_history, number_of_questions=100, TOEIC_strategy='random'):
    for i in range(number_of_questions):
        ### CHOIX DE LA QUESTION ###
        if TOEIC_strategy=='random':
            next_question_id=random.choice(qstats.content_id.to_list())


        user_history=user_history_update(0,
                                         next_question_id,
                                         qstats,
                                         user_history,
                                         mode='exam',
                                         prior_question_had_explanation=random.uniform(0, 1)>0.1)

        ### PREDICTION ###
        user_history.iloc[-1,-1]\
            = my_pipeline.predict_proba(user_history[pipeline_features_list.feature.to_list()].iloc[-2:-1])[0,1]
    
    return user_history.iloc[-number_of_questions:].answered_correctly.mean()

# Final exam

In [36]:
TOEIC_scoring(user_history)

0.7436589403231116

In [38]:
user_history=None
TOEIC_scoring(user_history)

0.6304356041477933